In [3]:

import sys
path = "C:\github/aide-analysis/"
sys.path.append(path)

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd())))

# 한글폰트
import os
import matplotlib.pyplot as plt
if os.name =='posix':
    plt.rc('font',family = 'ApplGothic')
else:
    plt.rc('font',family = 'Malgun Gothic')
    
# 경고출력문 제거
import warnings
warnings.filterwarnings("ignore")

# 모듈 import`
import pandas as pd
import src
from src.openapi import KakaoLocalAPI as kakao
from openpyxl import load_workbook
from src.utils import insert_dataframe_to_excel_sheet
from src.sql import DatabaseHandler
from src.temp import *
from datetime import datetime
import datetime
from dateutil.relativedelta import relativedelta
from src.bigquery import BigqueryHandler
import seaborn as sns
import numpy as np
import statistics
from tqdm import tqdm
import config
from core.config import Config
import json
from datetime import datetime
import geopandas as gpd
from tqdm import tqdm
import folium
from openpyxl import load_workbook
import openpyxl
import tqdm
config = Config()
BH = BigqueryHandler()


# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)


In [6]:
sql = f"""
SELECT *
FROM `aidepartners.aide.complex_real_price_prod` AS price
LEFT JOIN (SELECT * FROM `aidepartners.aide.complex_area_information_prod`) AS area ON price.complexNo = area.complexNo AND price.areaNo = area.pyeongNo
WHERE price.contractDate >='2020-01-01' AND tradeType = 'A1' AND deleteYn is null
"""
df = BH.read_table(sql)

In [51]:
data = df.copy()

In [52]:
# 년월 컬럼생성
data['yearMonth'] = data['tradeYear'].astype(str) + '-' +data['tradeMonth'].astype(str)
data['yearMonth'] = pd.to_datetime(data['yearMonth'])
# 매매거래 + 취소되지않은거래 필터
data = data[(data['tradeType']=='A1')&(data['deleteYn'].isna())].reset_index(drop = True)
# 월평균 거래금액 데이터프레임 생성
df_mean = data.groupby(['complexNo','yearMonth'])['dealPrice'].mean().reset_index()
# 월평균 거래금액 컬럼 명 변경
df_mean = df_mean.rename(columns = {'dealPrice':'averagePrice'})
# copy데이터에 월평균 거래금액 추가를 위한 merge
data = pd.merge(data, df_mean, on = ['complexNo','yearMonth'])

In [53]:
# 전월 평균 가격 컬럼 생성
data['beforePrice']

,complexNo,areaNo,tradeType,tradeYear,tradeMonth,tradeDate,dealPrice,leasePrice,rentPrice,floor,deleteYn,representativeArea,exclusiveArea,formattedPrice,contractDate,pnuCode,complexNo_1,complexName,realEstateTypeCode,realEstateTypeName,pyeongNo,supplyAreaName,supplyPyeongName,supplyArea,supplyPyeong,jeonyongArea,jeonyongPyeong,householdCountByPyeong,jeonyongRate,areaSixGroupNo,areaFiveGroupNo,entranceType,roomCount,bathroomCount,landPriceMaxByPtp,grandPlanList,articleStatistics,maintenanceCostList,averageMaintenanceCost,dealRestrictionYearMonthDay,monopolyPossibleYmd,isalePriceByLetter,isalePrice,stdYear,stdYmd,minPrice,maxPrice,minSupplyPrice,maxSupplyPrice,minJeonyongPrice,maxJeonyongPrice,imageSrc1,imageSrc2,isLedgerPrice,isPriceNull,isNearPrice,yearMonth,averagePrice
0,26532,1,A1,2022,4,1,5600,<NA>,<NA>,5,None,0.0,0.0,"5,600",2022-04-01,4374025031108860000,26532,영동설계주공,APT,아파트,1,55A,16A,55.93,16.91,39.94,12.08,106,71,A1,A0,복합식,2,1,"{'ptpNo': '1', 'supplyArea': '55.93', 'totalAr...",None,"{'pyeongNo': '1', 'dealCount': '1', 'leaseCoun...","[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '103800', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,3590,3970,212,235,297,329,None,None,0,0,0,2022-04-01,6728.571429
1,26532,2,A1,2022,4,6,5800,<NA>,<NA>,9,None,0.0,0.0,"5,800",2022-04-06,4374025031108860000,26532,영동설계주공,APT,아파트,2,55B,16B,55.51,16.79,39.64,11.99,54,71,A1,A0,복합식,2,1,"{'ptpNo': '2', 'supplyArea': '55.51', 'totalAr...",None,None,"[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '103020', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,3590,3970,214,236,299,331,None,None,0,0,0,2022-04-01,6728.571429
2,26532,2,A1,2022,4,8,6200,<NA>,<NA>,3,None,0.0,0.0,"6,200",2022-04-08,4374025031108860000,26532,영동설계주공,APT,아파트,2,55B,16B,55.51,16.79,39.64,11.99,54,71,A1,A0,복합식,2,1,"{'ptpNo': '2', 'supplyArea': '55.51', 'totalAr...",None,None,"[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '103020', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,3590,3970,214,236,299,331,None,None,0,0,0,2022-04-01,6728.571429
3,26532,3,A1,2022,4,21,7400,<NA>,<NA>,9,None,0.0,0.0,"7,400",2022-04-21,4374025031108860000,26532,영동설계주공,APT,아파트,3,69A,21A,69.75,21.09,49.81,15.06,197,71,A1,A1,복합식,2,1,"{'ptpNo': '3', 'supplyArea': '69.75', 'totalAr...",None,None,"[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '129451', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,4650,5430,220,257,309,361,None,None,0,0,0,2022-04-01,6728.571429
4,26532,3,A1,2022,4,5,8200,<NA>,<NA>,12,None,0.0,0.0,"8,200",2022-04-05,4374025031108860000,26532,영동설계주공,APT,아파트,3,69A,21A,69.75,21.09,49.81,15.06,197,71,A1,A1,복합식,2,1,"{'ptpNo': '3', 'supplyArea': '69.75', 'totalAr...",None,None,"[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '129451', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,4650,5430,220,257,309,361,None,None,0,0,0,2022-04-01,6728.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684619,24826,2,A1,2022,8,16,58850,<NA>,<NA>,27,None,0.0,0.0,"5억 8,850",2022-08-16,3111010800101720000,24826,마제스타워울산(주상복합),APT,아파트,2,174A,52A,174.91,52.91,133.51,40.38,197,76,A5,A4,계단식,4,2,"{'ptpNo': '2', 'supplyArea': '174.91', 'totalA...","[{'imageId': '24826@20120315163744934551', 'im...","{'pyeongNo': '2', 'dealCount': '13', 'leaseCou...","[{'basisYearMonth': '202210', 'totalPrice': '3...","{'averageTotalPrice': '305650', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,25400,36400,480,688,629,901,/20120315_91/hscp_img_13317970648546nBbN_JPEG/...,None,0,0,0,2022-08-01,58850.0
1684620,25380,6,A1,2022,8,21,35500,<NA>,<NA>,5,None,0.0,0.0,"3억 5,500",2022-08-21,4812710400102320001,25380,창원마산역동도센트리움(주상복합),APT,아파트,6,143,43,143.67,43.46,108.34,32.77,13,75,A4,A3,계단식

In [61]:
data['beforePrice'] = ''
for i in range(0, len(data)):
    data.iloc[i,-1] = data[data['yearMonth']==data['yearMonth'].iloc[i] - relativedelta(months = 1)]
    data['yearMonth'].iloc[i] - relativedelta(months = 1)

,complexNo,areaNo,tradeType,tradeYear,tradeMonth,tradeDate,dealPrice,leasePrice,rentPrice,floor,deleteYn,representativeArea,exclusiveArea,formattedPrice,contractDate,pnuCode,complexNo_1,complexName,realEstateTypeCode,realEstateTypeName,pyeongNo,supplyAreaName,supplyPyeongName,supplyArea,supplyPyeong,jeonyongArea,jeonyongPyeong,householdCountByPyeong,jeonyongRate,areaSixGroupNo,areaFiveGroupNo,entranceType,roomCount,bathroomCount,landPriceMaxByPtp,grandPlanList,articleStatistics,maintenanceCostList,averageMaintenanceCost,dealRestrictionYearMonthDay,monopolyPossibleYmd,isalePriceByLetter,isalePrice,stdYear,stdYmd,minPrice,maxPrice,minSupplyPrice,maxSupplyPrice,minJeonyongPrice,maxJeonyongPrice,imageSrc1,imageSrc2,isLedgerPrice,isPriceNull,isNearPrice,yearMonth,averagePrice
0,26532,1,A1,2022,4,1,5600,<NA>,<NA>,5,None,0.0,0.0,"5,600",2022-04-01,4374025031108860000,26532,영동설계주공,APT,아파트,1,55A,16A,55.93,16.91,39.94,12.08,106,71,A1,A0,복합식,2,1,"{'ptpNo': '1', 'supplyArea': '55.93', 'totalAr...",None,"{'pyeongNo': '1', 'dealCount': '1', 'leaseCoun...","[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '103800', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,3590,3970,212,235,297,329,None,None,0,0,0,2022-04-01,6728.571429
1,26532,2,A1,2022,4,6,5800,<NA>,<NA>,9,None,0.0,0.0,"5,800",2022-04-06,4374025031108860000,26532,영동설계주공,APT,아파트,2,55B,16B,55.51,16.79,39.64,11.99,54,71,A1,A0,복합식,2,1,"{'ptpNo': '2', 'supplyArea': '55.51', 'totalAr...",None,None,"[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '103020', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,3590,3970,214,236,299,331,None,None,0,0,0,2022-04-01,6728.571429
2,26532,2,A1,2022,4,8,6200,<NA>,<NA>,3,None,0.0,0.0,"6,200",2022-04-08,4374025031108860000,26532,영동설계주공,APT,아파트,2,55B,16B,55.51,16.79,39.64,11.99,54,71,A1,A0,복합식,2,1,"{'ptpNo': '2', 'supplyArea': '55.51', 'totalAr...",None,None,"[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '103020', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,3590,3970,214,236,299,331,None,None,0,0,0,2022-04-01,6728.571429
3,26532,3,A1,2022,4,21,7400,<NA>,<NA>,9,None,0.0,0.0,"7,400",2022-04-21,4374025031108860000,26532,영동설계주공,APT,아파트,3,69A,21A,69.75,21.09,49.81,15.06,197,71,A1,A1,복합식,2,1,"{'ptpNo': '3', 'supplyArea': '69.75', 'totalAr...",None,None,"[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '129451', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,4650,5430,220,257,309,361,None,None,0,0,0,2022-04-01,6728.571429
4,26532,3,A1,2022,4,5,8200,<NA>,<NA>,12,None,0.0,0.0,"8,200",2022-04-05,4374025031108860000,26532,영동설계주공,APT,아파트,3,69A,21A,69.75,21.09,49.81,15.06,197,71,A1,A1,복합식,2,1,"{'ptpNo': '3', 'supplyArea': '69.75', 'totalAr...",None,None,"[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '129451', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,4650,5430,220,257,309,361,None,None,0,0,0,2022-04-01,6728.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684619,24826,2,A1,2022,8,16,58850,<NA>,<NA>,27,None,0.0,0.0,"5억 8,850",2022-08-16,3111010800101720000,24826,마제스타워울산(주상복합),APT,아파트,2,174A,52A,174.91,52.91,133.51,40.38,197,76,A5,A4,계단식,4,2,"{'ptpNo': '2', 'supplyArea': '174.91', 'totalA...","[{'imageId': '24826@20120315163744934551', 'im...","{'pyeongNo': '2', 'dealCount': '13', 'leaseCou...","[{'basisYearMonth': '202210', 'totalPrice': '3...","{'averageTotalPrice': '305650', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,25400,36400,480,688,629,901,/20120315_91/hscp_img_13317970648546nBbN_JPEG/...,None,0,0,0,2022-08-01,58850.0
1684620,25380,6,A1,2022,8,21,35500,<NA>,<NA>,5,None,0.0,0.0,"3억 5,500",2022-08-21,4812710400102320001,25380,창원마산역동도센트리움(주상복합),APT,아파트,6,143,43,143.67,43.46,108.34,32.77,13,75,A4,A3,계단식

In [24]:
for i in range(0, len(data)):
    data[data['']]

,complexNo,areaNo,tradeType,tradeYear,tradeMonth,tradeDate,dealPrice,leasePrice,rentPrice,floor,deleteYn,representativeArea,exclusiveArea,formattedPrice,contractDate,pnuCode,complexNo_1,complexName,realEstateTypeCode,realEstateTypeName,pyeongNo,supplyAreaName,supplyPyeongName,supplyArea,supplyPyeong,jeonyongArea,jeonyongPyeong,householdCountByPyeong,jeonyongRate,areaSixGroupNo,areaFiveGroupNo,entranceType,roomCount,bathroomCount,landPriceMaxByPtp,grandPlanList,articleStatistics,maintenanceCostList,averageMaintenanceCost,dealRestrictionYearMonthDay,monopolyPossibleYmd,isalePriceByLetter,isalePrice,stdYear,stdYmd,minPrice,maxPrice,minSupplyPrice,maxSupplyPrice,minJeonyongPrice,maxJeonyongPrice,imageSrc1,imageSrc2,isLedgerPrice,isPriceNull,isNearPrice,averagePrice
0,26532,1,A1,2022,4,1,5600,<NA>,<NA>,5,None,0.0,0.0,"5,600",2022-04-01,4374025031108860000,26532,영동설계주공,APT,아파트,1,55A,16A,55.93,16.91,39.94,12.08,106,71,A1,A0,복합식,2,1,"{'ptpNo': '1', 'supplyArea': '55.93', 'totalAr...",None,"{'pyeongNo': '1', 'dealCount': '1', 'leaseCoun...","[{'basisYearMonth': '202209', 'totalPrice': '1...","{'averageTotalPrice': '103800', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,3590,3970,212,235,297,329,None,None,0,0,0,5600.0
1,13710,1,A1,2022,4,1,35000,<NA>,<NA>,18,None,0.0,0.0,"3억 5,000",2022-04-01,4213011500116400000,13710,요진보네르카운티,APT,아파트,1,115A,34A,115.17,34.83,86.69,26.22,379,75,A4,A3,계단식,3,2,None,"[{'imageId': '13710@20120224151650155465', 'im...","{'pyeongNo': '1', 'dealCount': '12', 'leaseCou...","[{'basisYearMonth': '202210', 'totalPrice': '1...","{'averageTotalPrice': '167187', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,16800,19800,482,568,641,755,/20120224_33/hscp_img_13300642100065tje3_JPEG/...,None,1,0,0,33300.0
2,13710,1,A1,2022,4,18,31800,<NA>,<NA>,6,None,0.0,0.0,"3억 1,800",2022-04-18,4213011500116400000,13710,요진보네르카운티,APT,아파트,1,115A,34A,115.17,34.83,86.69,26.22,379,75,A4,A3,계단식,3,2,None,"[{'imageId': '13710@20120224151650155465', 'im...","{'pyeongNo': '1', 'dealCount': '12', 'leaseCou...","[{'basisYearMonth': '202210', 'totalPrice': '1...","{'averageTotalPrice': '167187', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,16800,19800,482,568,641,755,/20120224_33/hscp_img_13300642100065tje3_JPEG/...,None,1,0,0,33300.0
3,13710,1,A1,2022,4,13,33000,<NA>,<NA>,6,None,0.0,0.0,"3억 3,000",2022-04-13,4213011500116400000,13710,요진보네르카운티,APT,아파트,1,115A,34A,115.17,34.83,86.69,26.22,379,75,A4,A3,계단식,3,2,None,"[{'imageId': '13710@20120224151650155465', 'im...","{'pyeongNo': '1', 'dealCount': '12', 'leaseCou...","[{'basisYearMonth': '202210', 'totalPrice': '1...","{'averageTotalPrice': '167187', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,16800,19800,482,568,641,755,/20120224_33/hscp_img_13300642100065tje3_JPEG/...,None,1,0,0,33300.0
4,13710,1,A1,2022,4,9,32000,<NA>,<NA>,7,None,0.0,0.0,"3억 2,000",2022-04-09,4213011500116400000,13710,요진보네르카운티,APT,아파트,1,115A,34A,115.17,34.83,86.69,26.22,379,75,A4,A3,계단식,3,2,None,"[{'imageId': '13710@20120224151650155465', 'im...","{'pyeongNo': '1', 'dealCount': '12', 'leaseCou...","[{'basisYearMonth': '202210', 'totalPrice': '1...","{'averageTotalPrice': '167187', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,16800,19800,482,568,641,755,/20120224_33/hscp_img_13300642100065tje3_JPEG/...,None,1,0,0,33300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684619,22444,13,A1,2021,10,1,235000,<NA>,<NA>,47,None,0.0,0.0,"23억 5,000",2021-10-01,2629010700109440000,22444,오륙도SK뷰,APT,아파트,13,294B,89B,294.31,89.02,223.24,67.53,6,76,A5,A4,계단식,4,3,"{'ptpNo': '13', 'supplyArea': '294.31', 'total...","[{'imageId': '22444@20110420222259556188', 'im...",None,"[{'basisYearMonth': '202210', 'totalPrice': '5...","{'averageTotalPrice': '536912', 'summerTotalPr...",None,None,None,<NA>,2022,20220101,174400,174400,1959,1959